<a href="https://colab.research.google.com/github/IdjiotSandwiches/knn-fer/blob/create-model/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dagshub
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.2/252.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/8

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
# import mlflow
# import dagshub
import os
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/facial-emotion-recognition"

Mounted at /content/drive
/content/drive/MyDrive/facial-emotion-recognition


In [ ]:
TRAIN_PATH = pathlib.Path('/train')
TEST_PATH = pathlib.Path('/test')
CLASS_NAME = os.listdir(TRAIN_PATH)
NUM_CLASSES = len(CLASS_NAME)
BATCH_SIZE = 32
IMAGE_SIZE = (48,48)
AUTOTUNE = tf.data.AUTOTUNE
COLOR_MODE = 'grayscale'
SEED = 42
SHUFFLE = True
VALIDATION_SPLIT = 0.2

In [ ]:
params = {
    "batch_size": BATCH_SIZE,
    "image_size": IMAGE_SIZE,
    "seed": SEED,
    "validation_split": VALIDATION_SPLIT,
    "color_mode": COLOR_MODE,
    "shuffle": SHUFFLE,
}

In [ ]:
train_images, train_labels = [], []

train_dir = os.listdir(TRAIN_PATH)
for i, label in enumerate(train_dir):
  path = os.listdir(f'{train_dir}/{label}')
  for img in path:
    train_images.append(img)
    train_labels.append(i)

In [ ]:
test_images, test_labels = [], []

test_dir = os.listdir(TEST_PATH)
for i, label in enumerate(test_dir):
  path = os.listdir(f'{test_dir}/{label}')
  for img in path:
    test_images.append(img)
    test_labels.append(i)